# Combining dataframes in Pandas with merge(), join, and concat()

In [3]:
# Create DataFrames
import pandas as pd
technologies = {
    'Courses':["Spark","PySpark","Python","pandas"],
    'Fee' :[20000,25000,22000,30000],
    'Duration':['30days','40days','35days','50days'],
              }
index_labels=['r1','r2','r3','r4']
df1 = pd.DataFrame(technologies,index=index_labels)

technologies2 = {
    'Courses':["Spark","Java","Python","Go"],
    'Discount':[2000,2300,1200,2000]
              }
index_labels2=['r1','r6','r3','r5']
df2 = pd.DataFrame(technologies2,index=index_labels2)
technologies3 = {
    'Instructor': ["Jerry", "Tom", "Smith", "John"],
    'Courses': ["Spark", "Java", "Python", "Scala"]
}
index_labels3 = ['r1','r6','r3','r7']
df3 = pd.DataFrame(technologies3, index=index_labels3)

print("First DataFrame:\n\n", df1)
print("\n")
print("Second DataFrame:\n\n", df2)
print("\n")
print("Third DataFrame:\n\n", df3)


First DataFrame:

     Courses    Fee Duration
r1    Spark  20000   30days
r2  PySpark  25000   40days
r3   Python  22000   35days
r4   pandas  30000   50days


Second DataFrame:

    Courses  Discount
r1   Spark      2000
r6    Java      2300
r3  Python      1200
r5      Go      2000


Third DataFrame:

    Instructor Courses
r1      Jerry   Spark
r6        Tom    Java
r3      Smith  Python
r7       John   Scala


## [.merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

In [5]:
df4 = pd.merge(df1,df2)
df4

,Courses,Fee,Duration,Discount
0,Spark,20000,30days,2000
1,Python,22000,35days,1200


If we just do a [merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) with no other parameters, it will attempt to merge the DataFrames on columns with the same name. Specifically, it will merge on columns that are common to both DataFrames. The default behavior is to perform an [inner join](https://blog.codinghorror.com/a-visual-explanation-of-sql-joins/) on the common columns. This can be neat if our data is clean like in this instance and we know things will line up well. It can be tricky though if we have columns that we don't intend to merge on (say if "Courses" meant an entirely different thing in our second data frame). 

We can of course specify the column we want to merge on. In this case, it will produce the same result. 

In [9]:
df4 = pd.merge(df1,df2, on = 'Courses')
df4

,Courses,Fee,Duration,Discount
0,Spark,20000,30days,2000
1,Python,22000,35days,1200


If we try to merge on an unshared column, we will get the following:

In [10]:
df4 = pd.merge(df1,df2, on ="Fee")

KeyError: 'Fee'

This error is telling us that the key "Fee" isn't present in one of our dataframes. And that is correct- it isn't present in our second one. 

Notice thus far that we have merged tables by calling <b> pd </b>'s method called merge(). We can alternatively call it merge() on the actual dataframe itself. 

In [11]:
df4 = df1.merge(df2)
df4

,Courses,Fee,Duration,Discount
0,Spark,20000,30days,2000
1,Python,22000,35days,1200


In [12]:
df4 = df1.merge(df2, on="Courses")
df4

,Courses,Fee,Duration,Discount
0,Spark,20000,30days,2000
1,Python,22000,35days,1200


We can change our merges too to bring in even more tables. 

In [13]:
df5 = df1.merge(df2).merge(df3)
df5

,Courses,Fee,Duration,Discount,Instructor
0,Spark,20000,30days,2000,Jerry
1,Python,22000,35days,1200,Smith


There are many different types of merges, we can specify if we want it to do a full outer join, right outer join, or left outer join. 

In [17]:
left = df1.merge(df2, how='left')
left

,Courses,Fee,Duration,Discount
0,Spark,20000,30days,2000.0
1,PySpark,25000,40days,NaN
2,Python,22000,35days,1200.0
3,pandas,30000,50days,NaN


Notice that both tables 1 and 2 were merged, but that PySpark and pandas didn't have discount values in the second table, so they get NaN (not a number) values. These are defaults for any [missing values](https://pandas.pydata.org/docs/user_guide/missing_data.html) in a pandas dataframe, especially after a merge. 

Notice also that the rows Java and Go do not appear at all in the resulting table. This is because a left join only takes the data in the left table (df1) and matches it with matches in the second table (df2) and fills in NaN's where applicable. It does not take the entries in df2 and match it with those in the left table. We will see the opposite in the right join. 

In [18]:
right = df1.merge(df2,how="right")
right

,Courses,Fee,Duration,Discount
0,Spark,20000.0,30days,2000
1,Java,NaN,NaN,2300
2,Python,22000.0,35days,1200
3,Go,NaN,NaN,2000


Now you can see that Java and Go have entries (since they are present in the right table (df2), but PySpark and Pandas are not present because they do not have entries in the right table at all. 

What about full outer join?

In [21]:
full = df1.merge(df2, how="outer")
full

,Courses,Fee,Duration,Discount
0,Spark,20000.0,30days,2000.0
1,PySpark,25000.0,40days,NaN
2,Python,22000.0,35days,1200.0
3,pandas,30000.0,50days,NaN
4,Java,NaN,NaN,2300.0
5,Go,NaN,NaN,2000.0


You see that it encapsulates all the entries, placing missing NaN values where they belong. <br> Java and Go are present from the second table, and PySpark and Pandas are present from the first table. <br> We didn't filter out for those values where NaN is present in the row. <br> <br>  If we did, we'd only get back two rows: Spark and Python..which gets us back to the inner join we started this activity with. <br> <br>
What if we wanted the records <b> unique </b> to df1? We can use our left join and filter for rows that have NaN for the discount. We know these won't be present in table 2, otherwise they would have a discount value.

In [24]:
unique_to_df1 = left[left['Discount'].isnull()]
unique_to_df1

,Courses,Fee,Duration,Discount
1,PySpark,25000,40days,NaN
3,pandas,30000,50days,NaN


We can do a similar thing to get the rows unique to df2. 

In [26]:
unique_to_df2 = right[right['Duration'].isnull()]
unique_to_df2

,Courses,Fee,Duration,Discount
1,Java,NaN,NaN,2300
3,Go,NaN,NaN,2000


Notice that I changed <b>Discount</b> to <b>Duration</b>, because in the right join every value had a Discount value, so our result would be an empty dataframe. In the right join though, some rows had NaN for the Fee and Duration. Those are the rows unique to the right table.

## [.join()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html)

Most of the time, you will use merge(), join() exists to make things a bit more readable for very simple joins. By default, join() does a left join, but you can specify using how= same as merge(). 

The caveat for .join() is that when you call it you want to set your index column (the column that uniquely identifies each row) before doing the join. 

In [27]:
join_example = df1.join(df2)
join_example

ValueError: columns overlap but no suffix specified: Index(['Courses'], dtype='object')

See, it gives me an error because I never set my index columns. 

In [28]:
df1.set_index('Courses', inplace=True)
df2.set_index('Courses', inplace=True)
df3.set_index('Courses', inplace=True)
join_example = df1.join(df2)
join_example

,Fee,Duration,Discount
Courses,,,
Spark,20000,30days,2000.0
PySpark,25000,40days,NaN
Python,22000,35days,1200.0
pandas,30000,50days,NaN


Now that I've set my indices, join will do the operation for me. Notice that it looks the same as our left join from earlier. 

In [30]:
left = df1.merge(df2, how='left', on = "Courses")
left

,Fee,Duration,Discount
Courses,,,
Spark,20000,30days,2000.0
PySpark,25000,40days,NaN
Python,22000,35days,1200.0
pandas,30000,50days,NaN


## [.concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)

Concatenation is a bit different. 
With concatenation, your datasets are just appended together along an axis — either along rows or columns. I've included some visuals in the folder for this activity that shows what a concatetation looks like. For the sake of our demonstration, I'm going to show a row concatetation. First, I'll create one more dataframe.

In [55]:
technologies4 = {
    'Courses':["C","C++","C#","Javascript"],
    'Fee' :[10000,15000,12000,40000],
    'Duration':['10days','20days','25days','50days'],
              }
df4 = pd.DataFrame(technologies4)
# I set the index to courses because I did that for df1 earlier in this notebook
# and by default pandas will create a new index (numerical) for me. I want to make sure these tables align when I concatenate. 
df4.set_index("Courses", inplace=True)
df4.head()

,Fee,Duration
Courses,,
C,10000,10days
C++,15000,20days
C#,12000,25days
Javascript,40000,50days


Now I will concatenate the rows from this fourth dataframe to our first dataframe. 

In [52]:
concatenated_df = pd.concat([df1,df4], axis='rows')
concatenated_df

,Fee,Duration
Courses,,
Spark,20000,30days
PySpark,25000,40days
Python,22000,35days
pandas,30000,50days
C,10000,10days
C++,15000,20days
C#,12000,25days
Javascript,40000,50days
